In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.0/297.0 kB 2.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 2.5 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 5.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for ipdb: filename=ipdb-0.13.9-py3-none-any.whl size=11628 sha256=c6dd89ff914e7cd6136698d0a27a7f8d7d683f9e934e9cb979893725092bc452
  Stored in directory: /home/jovyan/.cache/pip/wheels/54/02/23/574616a4405110c3a9ccf58c81a7bdea1145360f54adb240a5
Successfully built ipdb


In [14]:
from main import create_exchange
from lp import Factory

fabrica = Factory("Fábrica de piscinas de liquidez", "0x1")
piscinal_de_liquidez = create_exchange(name="test-coin", address="0x111", symbol="TST1", _factory=fabrica)

In [15]:
piscinal_de_liquidez.info()
piscinal_de_liquidez.doc()

Exchange test-coin/ETH (TST1)
Moedas: test-coin/ETH
Reservas: test-coin = 0 | ETH = 0
Invariante: 0 * 0 = 0

Funcionalidades disponíveis:
- Adicionar liquidez
- Remover liquidez
- Trocar tokens



In [16]:
eth_amount = 50000
tst_amount = 50000
piscinal_de_liquidez.add_liquidity(
    _from="rsarai",
    balance0=eth_amount,
    balance1=tst_amount,
    balance0Min=eth_amount,
    balance1Min=tst_amount
)
piscinal_de_liquidez.info()

Exchange test-coin/ETH (TST1)
Moedas: test-coin/ETH
Reservas: test-coin = 50000 | ETH = 50000
Invariante: 50000 * 50000 = 2500000000



In [17]:
piscinal_de_liquidez.simulate_transaction(amount_t0=50)
piscinal_de_liquidez.simulate_transaction(amount_t0=90)
piscinal_de_liquidez.simulate_transaction(amount_t0=300)
piscinal_de_liquidez.simulate_transaction(amount_t0=1000)
piscinal_de_liquidez.simulate_transaction(amount_t0=50000)

50 test-coin recebe 49.95 ETH
90 test-coin recebe 89.84 ETH
300 test-coin recebe 298.21 ETH
1000 test-coin recebe 980.39 ETH
50000 test-coin recebe 25000.0 ETH


In [18]:
piscinal_de_liquidez.swapExactTokensForTokens(amount0_in=50, amount1_out_min=375, to="rsarai")

AssertionError: UniswapV2Router: INSUFFICIENT_OUTPUT_AMOUNT

In [19]:
piscinal_de_liquidez.swapExactTokensForTokens(amount0_in=50, amount1_out_min=50, to="rsarai")

AssertionError: UniswapV2Router: INSUFFICIENT_OUTPUT_AMOUNT

In [20]:
print(piscinal_de_liquidez.swapExactTokensForTokens(amount0_in=50, amount1_out_min=49, to="rsarai"), '\n')
piscinal_de_liquidez.info()

49.95004995004995 

Exchange test-coin/ETH (TST1)
Moedas: test-coin/ETH
Reservas: test-coin = 50050 | ETH = 49950.04995004995
Invariante: 50050 * 49950.04995004995 = 2500000000.0



In [21]:
piscinal_de_liquidez.simulate_transaction(amount_t0=400)    # 396.04 ETH
print(piscinal_de_liquidez.swapExactTokensForTokens(amount0_in=400, amount1_out_min=396, to="rsarai"))
piscinal_de_liquidez.info()

400 test-coin recebe 396.04 ETH
396.03607492606506
Exchange test-coin/ETH (TST1)
Moedas: test-coin/ETH
Reservas: test-coin = 50450 | ETH = 49554.013875123885
Invariante: 50450 * 49554.013875123885 = 2500000000.0



In [22]:
piscinal_de_liquidez.simulate_transaction(amount_t0=400)    # 389.81 ETH
print(piscinal_de_liquidez.swapExactTokensForTokens(amount0_in=400, amount1_out_min=389, to="rsarai"))
piscinal_de_liquidez.info()

400 test-coin recebe 389.81 ETH
389.8054188800305
Exchange test-coin/ETH (TST1)
Moedas: test-coin/ETH
Reservas: test-coin = 50850 | ETH = 49164.208456243854
Invariante: 50850 * 49164.208456243854 = 2500000000.0



# Important code to check
- get_amount_out
- ERC20